In [9]:
import math
import cv2
import random as rd
import numpy as np  


def draw_keypoints_and_match(img1, img2):
    """This function is used for finding keypoints and dercriptors in the image and 
        find best matches using brute force/FLANN based matcher."""

    # Note : Can use sift too to improve feature extraction, but it can be patented again so it could brake the code in future!
    # Note: ORB is not scale independent so number of keypoints depend on scale
    # Initiate ORB detector
    orb = cv2.ORB_create(nfeatures=10000)
    # find the keypoints and descriptors with ORB
    kp1, des1 = orb.detectAndCompute(img1,None)
    kp2, des2 = orb.detectAndCompute(img2,None)

    #________________________Brute Force Matcher_____________________________
    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Match descriptors.
    matches = bf.match(des1,des2)

    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)
    # print(len(matches))
    
    # Select first 30 matches.
    final_matches = matches[:30]

    #___________________________________________________________________________

    #________________________FLANN based Matcher________________________________
    # FLANN_INDEX_LSH = 6
    # index_params = dict(
    #     algorithm=FLANN_INDEX_LSH,
    #     table_number=6,  # 12
    #     key_size=12,  # 20
    #     multi_probe_level=1,
    # )  # 2
    # search_params = dict(checks=50)  # or pass empty dictionary
    # flann = cv2.FlannBasedMatcher(index_params, search_params)
    # flann_match_pairs = flann.knnMatch(des1, des2, k=2)
    
    # # Filter matches using the Lowe's ratio test
    # ratio_threshold = 0.3
    # filtered_matches = []
    # for m, n in flann_match_pairs:
    #     if m.distance < ratio_threshold * n.distance:
    #         filtered_matches.append(m)
    
    # print("FMatches", len(filtered_matches))
    # final_matches =  filtered_matches[:100]
    #___________________________________________________________________________


    # Draw keypoints
    img_with_keypoints = cv2.drawMatches(img1,kp1,img2,kp2,final_matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv2.imwrite("images_with_matching_keypoints.png", img_with_keypoints)

    # Getting x,y coordinates of the matches
    list_kp1 = [list(kp1[mat.queryIdx].pt) for mat in final_matches] 
    list_kp2 = [list(kp2[mat.trainIdx].pt) for mat in final_matches]

    return list_kp1, list_kp2




def calculate_F_matrix(list_kp1, list_kp2):
    """This function is used to calculate the F matrix from a set of 8 points using SVD.
        Furthermore, the rank of F matrix is reduced from 3 to 2 to make the epilines converge."""

    A = np.zeros(shape=(len(list_kp1), 9))

    for i in range(len(list_kp1)):
        x1, y1 = list_kp1[i][0], list_kp1[i][1]
        x2, y2 = list_kp2[i][0], list_kp2[i][1]
        A[i] = np.array([x1*x2, x1*y2, x1, y1*x2, y1*y2, y1, x2, y2, 1])

    U, s, Vt = np.linalg.svd(A)
    F = Vt[-1,:]
    F = F.reshape(3,3)
   
    # Downgrading the rank of F matrix from 3 to 2
    Uf, Df, Vft = np.linalg.svd(F)
    Df[2] = 0
    s = np.zeros((3,3))
    for i in range(3):
        s[i][i] = Df[i]

    F = np.dot(Uf, np.dot(s, Vft))
    return F

def RANSAC_F_matrix(list_of_cood_list):
    """This method is used to shortlist the best F matrix using RANSAC based on the number of inliers."""
    
    list_kp1 = list_of_cood_list[0]
    list_kp2 = list_of_cood_list[1]
    pairs = list(zip(list_kp1, list_kp2))  
    max_inliers = 20
    threshold = 0.05  # Tune this value
  
    for i in range(1000):
        pairs = rd.sample(pairs, 8)  
        rd_list_kp1, rd_list_kp2 = zip(*pairs) 
        F = calculate_F_matrix(rd_list_kp1, rd_list_kp2)
        
        tmp_inliers_img1 = []
        tmp_inliers_img2 = []

        for i in range(len(list_kp1)):
            img1_x = np.array([list_kp1[i][0], list_kp1[i][1], 1])
            img2_x = np.array([list_kp2[i][0], list_kp2[i][1], 1])
            distance = abs(np.dot(img2_x.T, np.dot(F,img1_x)))
            # print(distance)

            if distance < threshold:
                tmp_inliers_img1.append(list_kp1[i])
                tmp_inliers_img2.append(list_kp2[i])

        num_of_inliers = len(tmp_inliers_img1)
        
        # if num_of_inliers > inlier_count:
        #     inlier_count = num_of_inliers
        #     Best_F = F

        if num_of_inliers > max_inliers:
            print("Number of inliers", num_of_inliers)
            max_inliers = num_of_inliers
            Best_F = F
            inliers_img1 = tmp_inliers_img1
            inliers_img2 = tmp_inliers_img2
            # print("Best F matrix", Best_F)

    return Best_F


def calculate_E_matrix(F, K1, K2):
    """Calculation of Essential matrix"""
    
    E = np.dot(K2.T, np.dot(F,K1))
    return E


def extract_camerapose(E):
    """This function extracts all the camera pose solutions from the E matrix"""

    U, s, Vt = np.linalg.svd(E)
    W = np.array([[0,-1, 0],
                  [1, 0, 0],
                  [0, 0, 1]])
    
    C1, C2 = U[:, 2], -U[:, 2]
    R1, R2 = np.dot(U, np.dot(W,Vt)), np.dot(U, np.dot(W.T, Vt))
    # print("C1", C1, "\n", "C2", C2, "\n", "R1", R1, "\n", "R2", R2, "\n")
    
    camera_poses = [[R1, C1], [R1, C2], [R2, C1], [R2, C2]]
    return camera_poses
    

def disambiguate_camerapose(camera_poses, list_kp1):
    """This fucntion is used to find the correct camera pose based on the chirelity condition from all 4 solutions."""

    max_len = 0
    # Calculating 3D points 
    for pose in camera_poses:

        front_points = []        
        for point in list_kp1:
            # Chirelity check
            X = np.array([point[0], point[1], 1])
            V = X - pose[1]
            
            condition = np.dot(pose[0][2], V)
            if condition > 0:
                front_points.append(point)    

        if len(front_points) > max_len:
            max_len = len(front_points)
            best_camera_pose =  pose
    
    return best_camera_pose
    

def drawlines(img1src, img2src, lines, pts1src, pts2src):
    """This fucntion is used to visualize the epilines on the images
        img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines """
    r, c = img1src.shape
    img1color = cv2.cvtColor(img1src, cv2.COLOR_GRAY2BGR)
    img2color = cv2.cvtColor(img2src, cv2.COLOR_GRAY2BGR)
    # Edit: use the same random seed so that two images are comparable!
    np.random.seed(0)
    for r, pt1, pt2 in zip(lines, pts1src, pts2src):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2]/r[1]])
        x1, y1 = map(int, [c, -(r[2]+r[0]*c)/r[1]])
        img1color = cv2.line(img1color, (x0, y0), (x1, y1), color, 1)
        img1color = cv2.circle(img1color, tuple(pt1), 5, color, -1)
        img2color = cv2.circle(img2color, tuple(pt2), 5, color, -1)
    
    return img1color, img2color

In [10]:
import numpy as np
import cv2

def sum_of_squared_diff(pixel_vals_1, pixel_vals_2):
    """Sum of squared distances for Correspondence"""
    if pixel_vals_1.shape != pixel_vals_2.shape:
        return -1

    return np.sum((pixel_vals_1 - pixel_vals_2)**2)

def block_comparison(y, x, block_left, right_array, block_size, x_search_block_size, y_search_block_size):
    """Block comparison function used for comparing windows on left and right images and find the minimum value ssd match the pixels"""
    
    # Get search range for the right image
    x_min = max(0, x - x_search_block_size)
    x_max = min(right_array.shape[1], x + x_search_block_size)
    y_min = max(0, y - y_search_block_size)
    y_max = min(right_array.shape[0], y + y_search_block_size)
    
    first = True
    min_ssd = None
    min_index = None

    for y in range(y_min, y_max):
        for x in range(x_min, x_max):
            block_right = right_array[y: y+block_size, x: x+block_size]
            ssd = sum_of_squared_diff(block_left, block_right)
            if first:
                min_ssd = ssd
                min_index = (y, x)
                first = False
            else:
                if ssd < min_ssd:
                    min_ssd = ssd
                    min_index = (y, x)

    return min_index


def ssd_correspondence(img1, img2):
    """Correspondence applied on the whole image to compute the disparity map and finally disparity map is scaled"""
    # Don't search full line for the matching pixel
    # grayscale imges

    block_size = 15 # 15
    x_search_block_size = 50 # 50 
    y_search_block_size = 1
    h, w = img1.shape
    disparity_map = np.zeros((h, w))

    for y in range(block_size, h-block_size):
        for x in range(block_size, w-block_size):
            block_left = img1[y:y + block_size, x:x + block_size]
            index = block_comparison(y, x, block_left, img2, block_size, x_search_block_size, y_search_block_size)
            disparity_map[y, x] = abs(index[1] - x)
    
    disparity_map_unscaled = disparity_map.copy()

    # Scaling the disparity map
    max_pixel = np.max(disparity_map)
    min_pixel = np.min(disparity_map)

    for i in range(disparity_map.shape[0]):
        for j in range(disparity_map.shape[1]):
            disparity_map[i][j] = int((disparity_map[i][j]*255)/(max_pixel-min_pixel))
    
    disparity_map_scaled = disparity_map
    return disparity_map_unscaled, disparity_map_scaled

In [11]:
import numpy as np
import cv2

def disparity_to_depth(baseline, f, img):
    """This is used to compute the depth values from the disparity map"""

    # Assumption image intensities are disparity values (x-x') 
    depth_map = np.zeros((img.shape[0], img.shape[1]))
    depth_array = np.zeros((img.shape[0], img.shape[1]))

    for i in range(depth_map.shape[0]):
        for j in range(depth_map.shape[1]):
            depth_map[i][j] = 1/img[i][j]
            depth_array[i][j] = baseline*f/img[i][j]
            # if math.isinf(depth_map[i][j]):
            #     depth_map[i][j] = 1

    return depth_map, depth_array

In [12]:
import math
import cv2
import random as rd
import numpy as np  
import matplotlib.pyplot as plt

from calibration import draw_keypoints_and_match, drawlines, RANSAC_F_matrix, calculate_E_matrix, extract_camerapose, disambiguate_camerapose
from rectification import rectification
from correspondence import ssd_correspondence
from depth import disparity_to_depth

# Its all about resolution - Its a trade off between resolution and time of computation

def main():
    number = int(input("Please enter the dataset number (1/2/3) to use for calculating the depth map\n"))
    img1 = cv2.imread(f"Dataset{number}/im0.png", 0)
    img2 = cv2.imread(f"Dataset{number}/im1.png", 0)

    width = int(img1.shape[1]* 0.3) # 0.3
    height = int(img1.shape[0]* 0.3) # 0.3

    img1 = cv2.resize(img1, (width, height), interpolation = cv2.INTER_AREA)
    # img1 = cv2.GaussianBlur(img1,(5,5),0)
    img2 = cv2.resize(img2, (width, height), interpolation = cv2.INTER_AREA)
    # img2 = cv2.GaussianBlur(img2,(5,5),0)
    
    #__________________Camera Parameters________________________________
    K11 = np.array([[5299.313,  0,   1263.818], 
                [0,      5299.313, 977.763],
                [0,          0,       1   ]])
    K12 = np.array([[5299.313,   0,    1438.004],
                [0,      5299.313,  977.763 ],
                [0,           0,      1     ]])

    K21 = np.array([[4396.869, 0, 1353.072],
                    [0, 4396.869, 989.702],
                    [0, 0, 1]])
    K22 = np.array([[4396.869, 0, 1538.86],
                [0, 4396.869, 989.702],
                [0, 0, 1]])
    
    K31 = np.array([[5806.559, 0, 1429.219],
                    [0, 5806.559, 993.403],
                    [ 0, 0, 1]])
    K32 = np.array([[5806.559, 0, 1543.51],
                    [ 0, 5806.559, 993.403],
                    [ 0, 0, 1]])
    camera_params = [(K11, K12), (K21, K22), (K31, K32)]

    while(1):
        try:
            list_kp1, list_kp2 = draw_keypoints_and_match(img1, img2)
            
            #_______________________________Calibration_______________________________

            F = RANSAC_F_matrix([list_kp1, list_kp2])
            print("F matrix", F)
            print("=="*20, '\n')
            K1, K2 = camera_params[number-1]
            E = calculate_E_matrix(F, K1, K2)
            print("E matrix", E)
            print("=="*20, '\n')
            camera_poses = extract_camerapose(E)
            best_camera_pose = disambiguate_camerapose(camera_poses, list_kp1)
            print("Best_Camera_Pose:")
            print("=="*20)
            print("Roatation", best_camera_pose[0])
            print()
            print("Transaltion", best_camera_pose[1])
            print("=="*20, '\n')
            pts1 = np.int32(list_kp1)
            pts2 = np.int32(list_kp2)

            #____________________________Rectification________________________________
            
            rectified_pts1, rectified_pts2, img1_rectified, img2_rectified = rectification(img1, img2, pts1, pts2, F)
            break
        except Exception as e:
            # print("error", e)
            continue
    
    # Find epilines corresponding to points in right image (second image) and drawing its lines on left image
    
    lines1 = cv2.computeCorrespondEpilines(rectified_pts2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(-1, 3)
    img5, img6 = drawlines(img1_rectified, img2_rectified, lines1, rectified_pts1, rectified_pts2)

    # Find epilines corresponding to points in left image (first image) and drawing its lines on right image

    lines2 = cv2.computeCorrespondEpilines(rectified_pts1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)
    img3, img4 = drawlines(img2_rectified, img1_rectified, lines2, rectified_pts2, rectified_pts1)

    cv2.imwrite("left_image.png", img5)
    cv2.imwrite("right_image.png", img3)
    
    #____________________________Correspondance________________________________
    
    disparity_map_unscaled, disparity_map_scaled = ssd_correspondence(img1_rectified, img2_rectified)
    # cv2.imwrite(f"disparity_map_{number}.png", disparity_map_scaled)

    # img_n = cv2.normalize(src=disparity_map_scaled, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    # heatmap1 = cv2.applyColorMap(img_n, cv2.COLORMAP_HOT)
    # cv2.imwrite(f"disparity_heat_map_{number}.png", heatmap1)
    plt.figure(1)
    plt.title('Disparity Map Graysacle')
    plt.imshow(disparity_map_scaled, cmap='gray')
    plt.figure(2)
    plt.title('Disparity Map Hot')
    plt.imshow(disparity_map_scaled, cmap='hot')
    

    #________________________________Depth______________________________________
    baseline1, f1 = 177.288, 5299.313
    baseline2, f2 = 144.049, 4396.869
    baseline3, f3 = 174.019, 5806.559
    
    params = [(baseline1, f1), (baseline2, f2), (baseline3, f3)]
    baseline, f = params[number-1]
    depth_map, depth_array = disparity_to_depth(baseline, f, disparity_map_unscaled)
    
    plt.figure(3)
    plt.title('Depth Map Graysacle')
    plt.imshow(depth_map, cmap='gray')
    plt.figure(4)
    plt.title('Depth Map Hot')
    plt.imshow(depth_map, cmap='hot')
    plt.show()

    print("=="*20)
    # print("Depth values", depth_array)

    #____________________________________________________________________________

if __name__ == "__main__":
    main()
    

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
import numpy as np
import cv2

def rectification(img1, img2, pts1, pts2, F):
    """This function is used to rectify the images to make camera pose's parallel and thus make epiplines as horizontal.
        Since camera distortion parameters are not given we will use cv2.stereoRectifyUncalibrated(), instead of stereoRectify().
    """

    # Stereo rectification
    h1, w1 = img1.shape
    h2, w2 = img2.shape

    _, H1, H2 = cv2.stereoRectifyUncalibrated(np.float32(pts1), np.float32(pts2), F, imgSize=(w1, h1))
    print("H1",H1)
    print("H2",H2)

    rectified_pts1 = np.zeros((pts1.shape), dtype=int)
    rectified_pts2 = np.zeros((pts2.shape), dtype=int)

    # Rectify the feature points
    for i in range(pts1.shape[0]):
        source1 = np.array([pts1[i][0], pts1[i][1], 1])
        new_point1 = np.dot(H1, source1)
        new_point1[0] = int(new_point1[0]/new_point1[2])
        new_point1[1] = int(new_point1[1]/new_point1[2])
        new_point1 = np.delete(new_point1, 2)
        rectified_pts1[i] = new_point1

        source2 = np.array([pts2[i][0], pts2[i][1], 1])
        new_point2 = np.dot(H2, source2)
        new_point2[0] = int(new_point2[0]/new_point2[2])
        new_point2[1] = int(new_point2[1]/new_point2[2])
        new_point2 = np.delete(new_point2, 2)
        rectified_pts2[i] = new_point2

    # Rectify the images and save them
    img1_rectified = cv2.warpPerspective(img1, H1, (w1, h1))
    img2_rectified = cv2.warpPerspective(img2, H2, (w2, h2))
    
    cv2.imwrite("rectified_1.png", img1_rectified)
    cv2.imwrite("rectified_2.png", img2_rectified)
    
    return rectified_pts1, rectified_pts2, img1_rectified, img2_rectified


   